In [ ]:
import polars as pl

In [ ]:
eager_df = pl.read_csv("../nyc_cab_data.csv", n_rows=10_000_000)

In [ ]:
# Data Manipulation
# 1. drop nulls
# 2. change field data types - with_column

In [ ]:
# insights
# 1. Average trip time 
# 2. Average passenger fare, tips
# 3. 5 Taxis with higher trips
# 4. Day with more number of trips
